Action2要求：    

使用Google Colab编辑器，对MovieLens数据集进行评分预测，计算RMSE
（使用funkSVD, BiasSVD，SVD++）

In [13]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# import os
# os.chdir("/content/drive/My Drive/Colab Notebooks/")


In [15]:
# !/opt/bin/nvidia-smi

Fri Sep 18 07:40:00 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [16]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null !apt-get update -qq 2>&1 > /dev/null !apt-get -y install -qq google-drive-ocamlfuse fuse from google.colab import auth auth.authenticate_user() from oauth2client.client import GoogleCredentials creds = GoogleCredentials.get_application_default() import getpass !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} &1 | grep URL vcode = getpass.getpass() !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `apt-get install -y -qq software-properties-common python-software-properties module-init-tools !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null !apt-get update -qq 2>&1 > /dev/null !apt-get -y install -qq google-drive-ocamlfuse fuse from google.colab import auth auth.authenticate_user() from oauth2client.client import GoogleCredentials creds = GoogleCredentials.get_application_default() import getpass !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} &1 | grep URL vcode = getpass.getpass() !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}'


In [17]:
# !python --version

Python 3.6.9


In [ ]:
# !pip install surprise

In [ ]:
# from surprise import SVD # 该API等价于下面的 SVD API

In [18]:
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from surprise.prediction_algorithms import SVD, SVDpp 
from surprise.model_selection import KFold, cross_validate

# 数据加载

## 1.1 数据读取

In [19]:
df = pd.read_csv('./ratings.csv')

In [20]:
df.head(2)

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676


In [21]:
# 告诉文本阅读器，文本的格式是怎样的
# line_format必须指出user item rating
reader = Reader(line_format="user item rating timestamp",
               sep=',',
               skip_lines=1)

In [22]:
#加载数据
data = Dataset.load_from_file('./ratings.csv', reader=reader)

In [23]:
type(data)

surprise.dataset.DatasetAutoFolds

In [24]:
train_set = data.build_full_trainset()

In [25]:
type(train_set)

surprise.trainset.Trainset

## 1.2 认识数据集

In [37]:
df.shape

(1048575, 4)

In [38]:
# 数据集中共有7120名用户、14026部电影，共计有1048575个评分
train_set.n_users, train_set.n_items, train_set.n_ratings

(7120, 14026, 1048575)

In [39]:
# 评分表的最小和最大评分
train_set.rating_scale

(1, 5)

In [40]:
df['rating'].min(), df['rating'].max()

(0.5, 5.0)

可见，surprise中，返回的最大最小评分进行了四舍五入取整

In [41]:
df.min(axis=0)

userId               1.0
movieId              1.0
rating               0.5
timestamp    825499934.0
dtype: float64

In [42]:
# 返回输入的用户id是否在train_set中，True代表存在
train_set.knows_user(0)

True

In [43]:
# 返回输入的itemid是否在train_set中，True代表存在
train_set.knows_item(0)

True

以上3块代码说明，数据集中用户id和电影id最小值均从1开始，但在surprise读入的数据集中，二者的id均从0开始，是surprise数据集中的内部id。

In [44]:
# 返回defaultdict字典，其中包含形式为（user_inner_id，rating）的元组列表。键是item的内部ID
#{item_inner_id:[(user_inner_id:rating)]}。
# train_set.ir

In [45]:
# # 将数据集加载为(user, item, rating, timestamp)的列表
# # 该方法读入的数据集，用户id与原始表格一致
# datas_list = data.read_ratings('./ratings.csv')

In [46]:
# type(datas_list), len(datas_list)

In [47]:
# datas_list[:2]

# SVD的3种算法：

## 2.1 funkSVD算法

FunkSVD目标是让用矩阵乘积得到的评分和用户的实际评分残差尽可能的小，使用均方差作为损失函数，同时为了防止过拟合，加入了L2正则化项，来寻找最终的矩阵P、Q。
$$arg \underset{p_{i}q_{j}}{min} \sum _{i,j\varepsilon K}(m_{ij}-p_{i}^Tq_{j})^2+\lambda (\left \| p_{i} \right \|_{2}^2 + \left \| q_{j} \right \|_{2}^2)$$

In [48]:
# biased=False代表使用funkSVD算法
model = SVD(biased=False)

In [49]:
# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)

In [50]:
%%time
for trainset, testset in kf.split(data):
    # 在训练集上训练
    model.fit(trainset)
    # 为测试集生成预测
    predictions = model.test(testset)
    # 计算rmse
    accuracy.rmse(predictions, 
                  verbose=True) # 默认值为True，代表将打印计算值

RMSE: 0.8750
RMSE: 0.8697
RMSE: 0.8726
CPU times: user 2min 5s, sys: 314 ms, total: 2min 5s
Wall time: 2min 5s


In [51]:
trainset.knows_user(0)

True

In [52]:
%%time
pred = model.predict(uid='0', iid='1', r_ui=3.5, verbose=True)

user: 0          item: 1          r_ui = 3.50   est = 3.53   {'was_impossible': True, 'reason': 'User and item are unknown.'}
CPU times: user 172 µs, sys: 1e+03 ns, total: 173 µs
Wall time: 752 µs


In [53]:
%%time
pred = model.predict(uid='1', iid='2', r_ui=3.5, verbose=True)

user: 1          item: 2          r_ui = 3.50   est = 3.44   {'was_impossible': False}
CPU times: user 314 µs, sys: 2 µs, total: 316 µs
Wall time: 261 µs


- `model.predict()`计算给定用户id和项目id的评分。
- predict方法会将原始id转换为内部id，然后调用在每个派生类中定义的estimate方法。
- `'was_impossible': True`如果预测是不可能的(例如，因为用户和/或项目是未知的)，则根据default_prediction()设置预测。
- 如前面2块代码所示，给uid与iid参数传参时，传入真实id即可，方法内部会自己转为内部id进行预测。

## 2.2 BiasSVD算法

In [54]:
# biased=True代表使用BiasSVD算法,默认值为True
biassvd = SVD(biased=True)
biassvd

In [55]:
# 3折交叉验证实例
kf = KFold(n_splits=3)

In [56]:
%%time
for trainset, testset in kf.split(data):
    # 在训练集上训练
    biassvd.fit(trainset)
    # 给测试集生成预测
    predictions = biassvd.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.8442
RMSE: 0.8467
RMSE: 0.8455
CPU times: user 2min 5s, sys: 319 ms, total: 2min 5s
Wall time: 2min 5s


In [57]:
pred = biassvd.predict(uid=1, iid=2, r_ui=3.5, verbose=True)

user: 1          item: 2          r_ui = 3.50   est = 3.53   {'was_impossible': False}


In [69]:
biassvd.predict(uid='1', iid='2', r_ui=3.5, verbose=True)

user: 1          item: 2          r_ui = 3.50   est = 3.59   {'was_impossible': False}


Prediction(uid='1', iid='2', r_ui=3.5, est=3.591033519079823, details={'was_impossible': False})

给参数`uid`和`iid`传参时，必须传入str类型，否则预测结果将会不准。

## 2.3 SVD++算法

In [58]:
from surprise.model_selection.split import train_test_split

In [59]:
svdpp = SVDpp()

In [60]:
trainset,testset = train_test_split(data, 
                                    test_size=0.2,
                                    random_state=30)

In [61]:
type(trainset), type(testset)

(surprise.trainset.Trainset, list)

In [62]:
trainset.n_users

7120

In [63]:
testset[:2]

[('2511', '3814', 4.5), ('923', '3243', 3.0)]

In [64]:
%%time
svdpp.fit(trainset)

CPU times: user 58min 9s, sys: 316 ms, total: 58min 9s
Wall time: 58min 11s


In [65]:
%%time
predictions = svdpp.test(testset)

CPU times: user 1min 22s, sys: 1.12 s, total: 1min 23s
Wall time: 1min 21s


In [66]:
%%time
accuracy.rmse(predictions, verbose=True)

RMSE: 0.8211
CPU times: user 181 ms, sys: 0 ns, total: 181 ms
Wall time: 182 ms


0.821147080364167

In [67]:
pred = svdpp.predict(uid=1, iid=2, r_ui=3.5, verbose=True)

user: 1          item: 2          r_ui = 3.50   est = 3.53   {'was_impossible': False}


In [68]:
svdpp.predict(uid='1', iid='2', r_ui=3.5, verbose=True)

user: 1          item: 2          r_ui = 3.50   est = 3.68   {'was_impossible': False}


Prediction(uid='1', iid='2', r_ui=3.5, est=3.6772612335019317, details={'was_impossible': False})

# 3 总结

1. 从预测的rmse值来看，`funkSVD > BiasSVD > SVDpp`；说明三个算法确实是由左向右的优化，而且是有效的优化；
2. SVD++ 使用3折交叉验证耗时太长, 用了2h 5min 30s；不使用交叉验证时，SVD++耗时58min 9s；
3. 通过2.2和2.3中的对比发现：向`model.predict()`的参数`uid`和`iid`传参时，必须传入str类型，否则预测结果将会不准。由于对SVD++进行3折交叉验证时，未使用字符类型进行传参，导致预测结果都是3.53，不具有参考性。
4. Trainset类型的数据集中，它会自动生成从0开始的内部uid和iid，因此如果通过该类对象的实例方法查看时，需要留意；
5. `model.predict()`进行预测时，可以向它传入真实`uid`和`iid`的str，该算法会自己转换为内部id后进行预测；
6. 预测结果中，如果返回结果中有`{'was_impossible': True}`，如2.1中的预测结果，代表该用户和商品的组合并不存在，需要检查传入是否有误。
7. funkSVD算法的目标函数，看起来和核心课第四周的ALS的目标函数相似，但测试集的均方根误差RMSE的值ALS算法的更低些，ALS(rmse=0.8366), funkSVD(rmse=0.8726)。